In [148]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [149]:
AS= pd.read_csv('../data/JetBlue_Airways.csv',skiprows=7)


In [150]:
df= pd.DataFrame(AS)
print(len(df))

201423


In [151]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
1678,B6,01/03/2018,284.0,N959JB,MCO,00:09,00:00
1686,B6,01/03/2018,426.0,N623JB,TPA,21:15,00:00
1710,B6,01/03/2018,842.0,N579JB,SAV,12:20,00:00
1720,B6,01/03/2018,1042.0,N775JB,SAV,21:14,00:00
1732,B6,01/03/2018,1274.0,N561JB,CHS,12:19,00:00
...,...,...,...,...,...,...,...
194529,B6,12/18/2016,185.0,N292JB,ROC,07:02,00:00
194531,B6,12/18/2016,207.0,N329JB,PWM,13:23,00:00
194574,B6,12/18/2016,838.0,N592JB,BQN,09:00,00:00
194623,B6,12/18/2016,2401.0,N353JB,BUF,14:48,00:00


In [152]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,B6,01/01/2015,2.0,N633JB,FLL,11:00,11:40
1,B6,01/01/2015,4.0,N913JB,SJU,16:10,16:01
2,B6,01/01/2015,14.0,N659JB,LGB,05:32,05:07
3,B6,01/01/2015,16.0,N934JB,SFO,15:17,15:08
4,B6,01/01/2015,24.0,N923JB,LAX,14:24,14:13


In [153]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [154]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,B6,01/01/2015,2.0,N633JB,FLL,11:00,11:40
1,B6,01/01/2015,4.0,N913JB,SJU,16:10,16:01
2,B6,01/01/2015,14.0,N659JB,LGB,05:32,05:07
3,B6,01/01/2015,16.0,N934JB,SFO,15:17,15:08
4,B6,01/01/2015,24.0,N923JB,LAX,14:24,14:13


In [155]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,B6,2015-01-01,2.0,N633JB,FLL,11:00,11:40
1,B6,2015-01-01,4.0,N913JB,SJU,16:10,16:01
2,B6,2015-01-01,14.0,N659JB,LGB,05:32,05:07
3,B6,2015-01-01,16.0,N934JB,SFO,15:17,15:08
4,B6,2015-01-01,24.0,N923JB,LAX,14:24,14:13
...,...,...,...,...,...,...,...
201417,B6,2018-12-31,2701.0,N324JB,BUF,11:12,10:39
201418,B6,2018-12-31,2704.0,N506JB,SJU,07:22,07:13
201419,B6,2018-12-31,2717.0,N192JB,BOS,23:59,00:05
201420,B6,2018-12-31,2807.0,N309JB,PWM,07:05,07:19


In [156]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,B6,2015-01-01,2.0,N633JB,FLL,11:00,11:40,2015-01-01
1,B6,2015-01-01,4.0,N913JB,SJU,16:10,16:01,2015-01-01
2,B6,2015-01-01,14.0,N659JB,LGB,05:32,05:07,2015-01-01
3,B6,2015-01-01,16.0,N934JB,SFO,15:17,15:08,2015-01-01
4,B6,2015-01-01,24.0,N923JB,LAX,14:24,14:13,2015-01-01
...,...,...,...,...,...,...,...,...
201417,B6,2018-12-31,2701.0,N324JB,BUF,11:12,10:39,2018-12-31
201418,B6,2018-12-31,2704.0,N506JB,SJU,07:22,07:13,2018-12-31
201419,B6,2018-12-31,2717.0,N192JB,BOS,23:59,00:05,2018-12-31
201420,B6,2018-12-31,2807.0,N309JB,PWM,07:05,07:19,2018-12-31


In [157]:
df.loc[df['Scheduled Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
26622,B6,2018-02-17,1902.0,N958JB,FLL,24:00,23:56,2018-02-17
72320,B6,2018-05-05,606.0,N729JB,ORD,24:00,00:01,2018-05-05
72898,B6,2018-05-06,606.0,N636JB,ORD,24:00,03:24,2018-05-06
76440,B6,2018-05-12,606.0,N586JB,ORD,24:00,24:00,2018-05-12
77028,B6,2018-05-13,606.0,N603JB,ORD,24:00,00:15,2018-05-13
80608,B6,2018-05-19,606.0,N607JB,ORD,24:00,00:37,2018-05-19
81201,B6,2018-05-20,606.0,N805JB,ORD,24:00,23:43,2018-05-20
83667,B6,2018-05-24,1826.0,N608JB,TPA,24:00,00:01,2018-05-24
84286,B6,2018-05-25,1826.0,N805JB,TPA,24:00,00:06,2018-05-25
84832,B6,2018-05-26,606.0,N652JB,ORD,24:00,23:31,2018-05-26


In [158]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')
df['Scheduled Arrival Time']= df['Scheduled Arrival Time'].str.replace('24:00','00:00')


In [159]:
df.loc[df['Scheduled Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
26622,B6,2018-02-17,1902.0,N958JB,FLL,00:00,23:56,2018-02-17
72320,B6,2018-05-05,606.0,N729JB,ORD,00:00,00:01,2018-05-05
72898,B6,2018-05-06,606.0,N636JB,ORD,00:00,03:24,2018-05-06
76440,B6,2018-05-12,606.0,N586JB,ORD,00:00,00:00,2018-05-12
77028,B6,2018-05-13,606.0,N603JB,ORD,00:00,00:15,2018-05-13
80608,B6,2018-05-19,606.0,N607JB,ORD,00:00,00:37,2018-05-19
81201,B6,2018-05-20,606.0,N805JB,ORD,00:00,23:43,2018-05-20
83667,B6,2018-05-24,1826.0,N608JB,TPA,00:00,00:01,2018-05-24
84286,B6,2018-05-25,1826.0,N805JB,TPA,00:00,00:06,2018-05-25
84832,B6,2018-05-26,606.0,N652JB,ORD,00:00,23:31,2018-05-26


In [160]:
df.loc[df['Actual Arrival Time']=='00:00','Actual Arrival Date']+=dt.timedelta(days=1)
df.loc[df['Scheduled Arrival Time']=='00:00','Scheduled Date']+=dt.timedelta(days=1)

In [161]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [162]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [163]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [164]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,B6,2015-01-01,2.0,N633JB,FLL,11:00,11:40,2015-01-01,2015-01-01 11:40,2015-01-01 11:00
1,B6,2015-01-01,4.0,N913JB,SJU,16:10,16:01,2015-01-01,2015-01-01 16:01,2015-01-01 16:10
2,B6,2015-01-01,14.0,N659JB,LGB,05:32,05:07,2015-01-01,2015-01-01 05:07,2015-01-01 05:32
3,B6,2015-01-01,16.0,N934JB,SFO,15:17,15:08,2015-01-01,2015-01-01 15:08,2015-01-01 15:17
4,B6,2015-01-01,24.0,N923JB,LAX,14:24,14:13,2015-01-01,2015-01-01 14:13,2015-01-01 14:24
...,...,...,...,...,...,...,...,...,...,...
201417,B6,2018-12-31,2701.0,N324JB,BUF,11:12,10:39,2018-12-31,2018-12-31 10:39,2018-12-31 11:12
201418,B6,2018-12-31,2704.0,N506JB,SJU,07:22,07:13,2018-12-31,2018-12-31 07:13,2018-12-31 07:22
201419,B6,2018-12-31,2717.0,N192JB,BOS,23:59,00:05,2018-12-31,2018-12-31 00:05,2018-12-31 23:59
201420,B6,2018-12-31,2807.0,N309JB,PWM,07:05,07:19,2018-12-31,2018-12-31 07:19,2018-12-31 07:05


In [165]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
1678,B6,2018-01-03,284.0,N959JB,MCO,00:09,00:00,2018-01-04,2018-01-04 00:00,2018-01-03 00:09
1686,B6,2018-01-03,426.0,N623JB,TPA,21:15,00:00,2018-01-04,2018-01-04 00:00,2018-01-03 21:15
1710,B6,2018-01-03,842.0,N579JB,SAV,12:20,00:00,2018-01-04,2018-01-04 00:00,2018-01-03 12:20
1720,B6,2018-01-03,1042.0,N775JB,SAV,21:14,00:00,2018-01-04,2018-01-04 00:00,2018-01-03 21:14
1732,B6,2018-01-03,1274.0,N561JB,CHS,12:19,00:00,2018-01-04,2018-01-04 00:00,2018-01-03 12:19
...,...,...,...,...,...,...,...,...,...,...
197911,B6,2015-12-25,184.0,N503JB,MCO,00:06,00:00,2015-12-26,2015-12-26 00:00,2015-12-25 00:06
198033,B6,2016-12-25,184.0,N526JB,MCO,23:50,00:00,2016-12-26,2016-12-26 00:00,2016-12-25 23:50
198161,B6,2017-12-25,284.0,N956JB,MCO,00:09,00:00,2017-12-26,2017-12-26 00:00,2017-12-25 00:09
198329,B6,2018-12-25,1186.0,N821JB,RDU,21:18,00:00,2018-12-26,2018-12-26 00:00,2018-12-25 21:18


In [166]:

FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,B6,2015-01-01,2.0,N633JB,FLL,11:00,11:40,2015-01-01,2015-01-01 11:40:00,2015-01-01 11:00:00
1,B6,2015-01-01,4.0,N913JB,SJU,16:10,16:01,2015-01-01,2015-01-01 16:01:00,2015-01-01 16:10:00
2,B6,2015-01-01,14.0,N659JB,LGB,05:32,05:07,2015-01-01,2015-01-01 05:07:00,2015-01-01 05:32:00
3,B6,2015-01-01,16.0,N934JB,SFO,15:17,15:08,2015-01-01,2015-01-01 15:08:00,2015-01-01 15:17:00
4,B6,2015-01-01,24.0,N923JB,LAX,14:24,14:13,2015-01-01,2015-01-01 14:13:00,2015-01-01 14:24:00
...,...,...,...,...,...,...,...,...,...,...
201417,B6,2018-12-31,2701.0,N324JB,BUF,11:12,10:39,2018-12-31,2018-12-31 10:39:00,2018-12-31 11:12:00
201418,B6,2018-12-31,2704.0,N506JB,SJU,07:22,07:13,2018-12-31,2018-12-31 07:13:00,2018-12-31 07:22:00
201419,B6,2018-12-31,2717.0,N192JB,BOS,23:59,00:05,2018-12-31,2018-12-31 00:05:00,2018-12-31 23:59:00
201420,B6,2018-12-31,2807.0,N309JB,PWM,07:05,07:19,2018-12-31,2018-12-31 07:19:00,2018-12-31 07:05:00


In [167]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,B6,2015-01-01,2.0,N633JB,FLL,11:00,11:40,2015-01-01,2015-01-01 11:40:00,2015-01-01 11:00:00,-40.0
1,B6,2015-01-01,4.0,N913JB,SJU,16:10,16:01,2015-01-01,2015-01-01 16:01:00,2015-01-01 16:10:00,9.0
2,B6,2015-01-01,14.0,N659JB,LGB,05:32,05:07,2015-01-01,2015-01-01 05:07:00,2015-01-01 05:32:00,25.0
3,B6,2015-01-01,16.0,N934JB,SFO,15:17,15:08,2015-01-01,2015-01-01 15:08:00,2015-01-01 15:17:00,9.0
4,B6,2015-01-01,24.0,N923JB,LAX,14:24,14:13,2015-01-01,2015-01-01 14:13:00,2015-01-01 14:24:00,11.0
...,...,...,...,...,...,...,...,...,...,...,...
201417,B6,2018-12-31,2701.0,N324JB,BUF,11:12,10:39,2018-12-31,2018-12-31 10:39:00,2018-12-31 11:12:00,33.0
201418,B6,2018-12-31,2704.0,N506JB,SJU,07:22,07:13,2018-12-31,2018-12-31 07:13:00,2018-12-31 07:22:00,9.0
201419,B6,2018-12-31,2717.0,N192JB,BOS,23:59,00:05,2018-12-31,2018-12-31 00:05:00,2018-12-31 23:59:00,1434.0
201420,B6,2018-12-31,2807.0,N309JB,PWM,07:05,07:19,2018-12-31,2018-12-31 07:19:00,2018-12-31 07:05:00,-14.0


In [168]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2019
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)

Total Alaska Airlines flights that landed at JFK in 2019: 27805
Flights that arrived early in 2019: 17966
Flights that arrived more than 20 mins early in 2019: 7521


In [169]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2018
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)

Total Alaska Airlines flights that landed at JFK in 2018: 43196
Flights that arrived early in 2018: 24721
Flights that arrived more than 20 mins early in 2018: 7586


In [170]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2017
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)

Total Alaska Airlines flights that landed at JFK in 2017: 42660
Flights that arrived early in 2017: 25406
Flights that arrived more than 20 mins early in 2017: 9602


In [171]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2016
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)

Total Alaska Airlines flights that landed at JFK in 2016: 43693
Flights that arrived early in 2016: 25808
Flights that arrived more than 20 mins early in 2016: 6812


In [172]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2015
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

Total Alaska Airlines flights that landed at JFK in 2015: 44068
Flights that arrived early in 2015: 26937
Flights that arrived more than 20 mins early in 2015: 8420


In [173]:


# dictionary of lists  
 
year =["2015","2016","2017","2018","2019"]
total_flights= [Total_15, Total_16, Total_17, Total_18, Total_19]
early_arrival=[Early_Arrivals_15,Early_Arrivals_16,Early_Arrivals_17,Early_Arrivals_18,Early_Arrivals_19]
inflated_padding=[Twenty_Mins_Early_15,Twenty_Mins_Early_16,Twenty_Mins_Early_17,Twenty_Mins_Early_18,Twenty_Mins_Early_19]

dict = {'year': year, 'Total flights': total_flights, 'Early Arrivals': early_arrival, 'Inflated padding(>20mins early)': inflated_padding} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'../output/JetBlue_Airways.csv',index=False) 